In [138]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from warnings import filterwarnings
import pymysql
filterwarnings('ignore', category=pymysql.Warning)

import pandas as pd
import numpy as np
import csv
import os
import io
import csv, sqlite3

Using SQlite to query All Missouri csv file for mean math, reading, science and social scores filtered for 2017. This will be used to create grouped bar chart comparing Jackson and Clay county to rest of the state scores.

In [139]:
missouri_file_name = os.path.join('Resources', 'mo_district_state_assessment_results.csv')
missouri_file = pd.read_csv(missouri_file_name)
mdf = pd.DataFrame(missouri_file)
mdf



,YEAR,SUMMARY_LEVEL,CATEGORY,TYPE,COUNTY_DISTRICT,DISTRICT_NAME,CONTENT_AREA,BELOW_BASIC_PCT,BASIC_PCT,PROFICIENT_PCT,ADVANCED_PCT
0,2015,District,MSIP Race no A1 E2 vol EOC,Asian/Pacific Islander,48914,ACADEMIE LAFAYETTE,Science,0.0,0.0,60.0,0.0
1,2015,District,MSIP Race no vol EOC,Asian/Pacific Islander,48914,ACADEMIE LAFAYETTE,Science,0.0,0.0,60.0,0.0
2,2017,District,MSIP Spec Pgm no A1 E2 vol EOC,ELL Students,48914,ACADEMIE LAFAYETTE,Mathematics,0.0,50.0,37.5,0.0
3,2016,District,MSIP Spec Pgm no A1 E2 vol EOC,ELL Students,48914,ACADEMIE LAFAYETTE,Eng. Language Arts,0.0,40.0,40.0,0.0
4,2016,District,MSIP Spec Pgm no vol EOC,ELL Students,48914,ACADEMIE LAFAYETTE,Eng. Language Arts,0.0,40.0,40.0,0.0
5,2017,District,MSIP Race no A1 E2 vol EOC,Multiracial,48914,ACADEMIE LAFAYETTE,Science,0.0,40.0,33.3,0.0
6,2015,District,MSIP Spec Pgm no A1 E2 vol EOC,ELL Students,48914,ACADEMIE LAFAYETTE,Eng. Language Arts,0.0,33.3,38.9,0.0
7,2015,District,MSIP Spec Pgm no vol EOC,ELL Students,48914,ACADEMIE LAFAYETTE,Eng. Language Arts,0.0,33.3,38.9,0.0
8,2017,District,MSIP Race no A1 E2 vol EOC,Asian/Pacific Islander,48914,ACADEMIE LAFAYETTE,Mathematics,5.9,5.9,23.5,64.7
9,2017,District,MSIP Race no A1 E2 vol EOC,Asian/Pacific Islander,48914,ACADEMIE LAFAYETTE,Eng. Language Arts,0.0,0.0,33.3,61.1


In [140]:
# engine = create_engine("sqlite:/data/state_assessments.sqlite", echo=False)
# engine.execute("DROP TABLE IF EXISTS missouri")
# Base.metadata.create_all(engine)

In [141]:
# https://stackoverflow.com/questions/2887878/importing-a-csv-file-into-a-sqlite3-database-table-using-python
con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("CREATE TABLE t (YEAR, SUMMARY_LEVEL, CATEGORY, TYPE, COUNTY_DISTRICT, DISTRICT_NAME, CONTENT_AREA, BELOW_BASIC_PCT, BASIC_PCT, PROFICIENT_PCT, ADVANCED_PCT;") # use your column names here

with open('/Resources/mo_district_state_assessment_results.csv','rb') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    state_assessments = [(i['YEAR'], i['SUMMARY_LEVEL'], i['CATEGORY'], i['TYPE'], i['COUNTY_DISRICT'], i['DISTRICT_NAME'], i['CONTENT_AREA'], i['BELOW_BASIC_PCT'], i['BASIC_PCT'], i['PROFICIENT_PCT'], i['ADVANCED_PCT']) for i in dr]

cur.executemany("INSERT INTO t (YEAR, SUMMARY_LEVEL, CATEGORY, TYPE, COUNTY_DISTRICT, DISTRICT_NAME, CONTENT_AREA, BELOW_BASIC_PCT, BASIC_PCT, PROFICIENT_PCT, ADVANCED_PCT) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", state_assessments)
con.commit()
con.close()

OperationalError: near ";": syntax error

In [142]:
class Missouri_Results(Base):
    __tablename__ = 'missouri'
    id = Column(Integer, primary_key=True)
    YEAR = Column(Integer) 
    SUMMARY_LEVEL = Column(String)
    CATEGORY = Column(String)
    TYPE = Column(String)
    COUNTY_DISTRICT = Column(Integer)
    DISTRICT_NAME = Column(String)
    CONTENT_AREA = Column(String)
    BELOW_BASIC_PCT = Column(Float)
    BASIC_PCT = Column(Float)
    PROFICIENT_PCT = Column(Float)
    ADVANCED_PCT = Column(Float)

In [148]:
engine = create_engine("sqlite:///data/state_assessments.sqlite", echo=False)
engine.execute("DROP TABLE IF EXISTS state_assessments")
Base.metadata.create_all(engine)
session = Session(bind=engine)

In [149]:
sql_query = """
    SELECT *
    FROM missouri
    ;
"""
pd.read_sql_query(sql_query, engine)

,id,YEAR,SUMMARY_LEVEL,CATEGORY,TYPE,COUNTY_DISTRICT,DISTRICT_NAME,CONTENT_AREA,BELOW_BASIC_PCT,BASIC_PCT,PROFICIENT_PCT,ADVANCED_PCT
0,1,2015,District,MSIP Race no A1 E2 vol EOC,Asian/Pacific Islander,48914.0,ACADEMIE LAFAYETTE,Science,0.0,0.0,60.0,0.0
1,2,2015,District,MSIP Race no vol EOC,Asian/Pacific Islander,48914.0,ACADEMIE LAFAYETTE,Science,0.0,0.0,60.0,0.0
2,3,2017,District,MSIP Spec Pgm no A1 E2 vol EOC,ELL Students,48914.0,ACADEMIE LAFAYETTE,Mathematics,0.0,50.0,37.5,0.0
3,4,2016,District,MSIP Spec Pgm no A1 E2 vol EOC,ELL Students,48914.0,ACADEMIE LAFAYETTE,Eng. Language Arts,0.0,40.0,40.0,0.0
4,5,2016,District,MSIP Spec Pgm no vol EOC,ELL Students,48914.0,ACADEMIE LAFAYETTE,Eng. Language Arts,0.0,40.0,40.0,0.0
5,6,2017,District,MSIP Race no A1 E2 vol EOC,Multiracial,48914.0,ACADEMIE LAFAYETTE,Science,0.0,40.0,33.3,0.0
6,7,2015,District,MSIP Spec Pgm no A1 E2 vol EOC,ELL Students,48914.0,ACADEMIE LAFAYETTE,Eng. Language Arts,0.0,33.3,38.9,0.0
7,8,2015,District,MSIP Spec Pgm no vol EOC,ELL Students,48914.0,ACADEMIE LAFAYETTE,Eng. Language Arts,0.0,33.3,38.9,0.0
8,9,2017,District,MSIP Race no A1 E2 vol EOC,Asian/Pacific Islander,48914.0,ACADEMIE LAFAYETTE,Mathematics,5.9,5.9,23.5,64.7
9,10,2017,District,MSIP Race no A1 E2 vol EOC,Asian/Pacific Islander,48914.0,ACADEMIE LAFAYETTE,Eng. Language Arts,0.0,0.0,33.3,61.1


In [150]:
sql_query = """
Select *
FROM missouri
Where TYPE == ("Total") and YEAR == ("2017") and CONTENT_AREA == ("Eng. Language Arts")
"""
pd.read_sql_query(sql_query, engine)

,id,YEAR,SUMMARY_LEVEL,CATEGORY,TYPE,COUNTY_DISTRICT,DISTRICT_NAME,CONTENT_AREA,BELOW_BASIC_PCT,BASIC_PCT,PROFICIENT_PCT,ADVANCED_PCT
0,35,2017,District,MSIP Total no A1 E2 vol EOC,Total,48914.0,ACADEMIE LAFAYETTE,Eng. Language Arts,3.7,10.4,45.6,40.3
1,170,2017,District,MSIP Total no A1 E2 vol EOC,Total,48927.0,ACADEMY FOR INTEGRATED ARTS,Eng. Language Arts,48.2,23.5,25.9,2.4
2,210,2017,District,MSIP Total no A1 E2 vol EOC,Total,1090.0,ADAIR CO. R-I,Eng. Language Arts,16.4,25.0,42.3,16.4
3,305,2017,District,MSIP Total no A1 E2 vol EOC,Total,1092.0,ADAIR CO. R-II,Eng. Language Arts,11.3,27.5,46.3,15.0
4,361,2017,District,MSIP Total no A1 E2 vol EOC,Total,7123.0,ADRIAN R-III,Eng. Language Arts,8.9,12.3,47.6,31.2
5,468,2017,District,MSIP Total no A1 E2 vol EOC,Total,103129.0,ADVANCE R-IV,Eng. Language Arts,17.6,24.4,47.2,10.9
6,587,2017,District,MSIP Total no A1 E2 vol EOC,Total,96098.0,AFFTON 101,Eng. Language Arts,12.0,22.6,44.4,21.0
7,745,2017,District,MSIP Total no A1 E2 vol EOC,Total,38046.0,ALBANY R-III,Eng. Language Arts,12.1,28.4,43.2,16.3
8,852,2017,District,MSIP Total no A1 E2 vol EOC,Total,48909.0,ALLEN VILLAGE,Eng. Language Arts,16.5,23.4,48.4,11.7
9,1048,2017,District,MSIP Total no A1 E2 vol EOC,Total,48902.0,ALTA VISTA CHARTER SCH.,Eng. Language Arts,46.1,28.8,22.3,2.8


In [151]:
sql_query = """
Select avg(BELOW_BASIC_PCT), avg(BASIC_PCT), avg(PROFICIENT_PCT), avg(ADVANCED_PCT)
FROM missouri
Where TYPE == ("Total") and YEAR == ("2017") and CONTENT_AREA == ("Eng. Language Arts")

"""
pd.read_sql_query(sql_query, engine)

,avg(BELOW_BASIC_PCT),avg(BASIC_PCT),avg(PROFICIENT_PCT),avg(ADVANCED_PCT)
0,17.714388,22.192626,42.35018,17.632374


In [152]:
sql_query = """
Select avg(BELOW_BASIC_PCT), avg(BASIC_PCT), avg(PROFICIENT_PCT), avg(ADVANCED_PCT)
FROM missouri
Where TYPE == ("Total") and YEAR == ("2017") and CONTENT_AREA == ("Mathematics")

"""
pd.read_sql_query(sql_query, engine)

,avg(BELOW_BASIC_PCT),avg(BASIC_PCT),avg(PROFICIENT_PCT),avg(ADVANCED_PCT)
0,19.791741,35.500718,28.694794,15.758169


In [153]:
sql_query = """
Select avg(BELOW_BASIC_PCT), avg(BASIC_PCT), avg(PROFICIENT_PCT), avg(ADVANCED_PCT)
FROM missouri
Where TYPE == ("Total") and YEAR == ("2017") and CONTENT_AREA == ("Science")

"""
pd.read_sql_query(sql_query, engine)

,avg(BELOW_BASIC_PCT),avg(BASIC_PCT),avg(PROFICIENT_PCT),avg(ADVANCED_PCT)
0,11.555926,39.259259,36.096296,11.813148
